# MIMIC 4 data - dataset construction admissions

Code taken from GRU-ODE-Bayes preprocessing; simplified and adapted for MIMIC 4 1.0

In [1]:
import os
import pathlib

p_project = str(pathlib.Path(os.getcwd()).parents[1])

In [2]:
import os
import pandas as pd
from datetime import datetime
import numpy as np

In [3]:
path_data = p_project + '/data/original/mimic4'
path_temp = p_project + '/data/mimic4'

In [4]:
fn = path_data + '/core/admissions.csv.gz'
adm = pd.read_csv(fn, compression='gzip')

In [5]:
#keep only patients present in patients data
patients_df=pd.read_csv(path_data + '/core/patients.csv.gz')
adm_dob=pd.merge(patients_df[['subject_id','anchor_age']], adm, on='subject_id')

In [6]:
#keep only patients present in ICU
icustay_df = pd.read_csv(path_data + '/icu/icustays.csv.gz', compression='gzip')
adm_icu = pd.merge(icustay_df[['hadm_id', 'los']], adm_dob, on='hadm_id')
adm_icu.rename(columns = {'los': 'icu_los'}, inplace = True)

In [7]:
adm_icu_selected=adm_icu.loc[(adm_icu['icu_los']<30) & (adm_icu['icu_los']>1)]
print('Number of patients remaining in the dataframe: ')
print(len(adm_icu_selected.index))

Number of patients remaining in the dataframe: 
60043


In [8]:
adm_icu_selected.head()

,hadm_id,icu_los,subject_id,anchor_age,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,24528534,1.587454,17867402,25,2154-03-03 03:09:00,2154-03-04 16:30:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,NaN,UNABLE TO OBTAIN,2154-03-03 02:49:00,2154-03-03 04:11:00,0
1,28960964,3.025625,14435996,42,2150-06-19 13:07:00,2150-06-25 15:36:00,NaN,OBSERVATION ADMIT,TRANSFER FROM HOSPITAL,HOME,Medicaid,ENGLISH,SINGLE,WHITE,2150-06-19 11:54:00,2150-06-19 17:57:00,0
2,27385897,9.741725,17609946,70,2138-02-05 17:42:00,2138-02-15 11:00:00,2138-02-15 11:00:00,OBSERVATION ADMIT,EMERGENCY ROOM,DIED,Medicare,ENGLISH,SINGLE,WHITE,2138-02-05 15:44:00,2138-02-05 18:54:00,1
4,20817525,1.674769,12776735,72,2200-07-11 22:46:00,2200-07-19 12:00:00,NaN,OBSERVATION ADMIT,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,OTHER,2200-07-11 15:27:00,2200-07-12 00:33:00,0
5,24283593,1.292697,10215159,67,2124-09-20 15:04:00,2124-09-26 14:30:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicare,ENGLISH,DIVORCED,WHITE,2124-09-20 12:52:00,2124-09-20 17:21:00,0


In [9]:
path_processed = path_temp + '/processed'
if not os.path.exists(path_processed):
    os.mkdir(path_processed)

In [10]:
adm_icu_selected.to_csv(path_processed + '/admissions_processed.csv')